In [1]:
import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import os, re, csv
import pandas as pd
from collections import Counter

## 대전지방경찰청 교통정보센터 트위터 내용 크롤링하기

- 대전지방경찰청 교통정보센터를 크롤링 하면서 어느 지역의 교통이 붐비는지 언급된 내용을 count하여 교통안전이 필요한 도로를 확인할 수 있다.
- webdriver : chromedriver.exe 존재해야함. 

## 1. 스크롤 방식의 리스트에 넣기

In [2]:
text_list = [] # 뉴스기록을 넣을 빈 리스트 생성

In [3]:
def doScrollDown1(whileSeconds):    
    
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    
    url = "https://twitter.com/poltra042?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor"
    driver = webdriver.Chrome('C:\chromedriver\chromedriver.exe') 
    driver.implicitly_wait(30) 
    driver.get(url)
    
    SCROLL_PAUSE_TIME = 1
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 페이지 로드 대기 
        time.sleep(SCROLL_PAUSE_TIME) 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") 
        time.sleep(SCROLL_PAUSE_TIME) 
        
        texts = driver.find_elements_by_css_selector('span.css-901oao.css-16my406.r-poiln3.r-bcqeeo.r-qvutc0') 
    
        for item in texts:
            text_list.append(item.text)
        
        # 새로운 스크롤 동작 높이와 이전 스크롤과의 비교 
        new_height = driver.execute_script("return document.body.scrollHeight") 
        
        
        if new_height == last_height: 
            break
            
        last_height = new_height 
        
        if datetime.datetime.now() > end:
            break
            
    driver.quit()

In [4]:
doScrollDown1(600)

WebDriverException: Message: 'C:\chromedriver\chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


3.11일 부터 4.1일 까지 데이터

##  트위터의 경우에는 페이지가 넘어가는 형식이 아닌 무한 스크롤 방식이므로 웹크롤링할때 셀레니움 이라는 패키지를 사용해야한다.

In [ ]:
text_list

In [ ]:
obj = ['밀림','주의','지체']
texts = []

In [ ]:
for i in text_list:
    for j in obj:
        if j in i:
            texts.append(i)

각각 밀림, 주의 지체등 유형에 따라 분리

In [ ]:
texts

,를 기준으로 나누기

In [ ]:
장소 = []
방향 = []
유형 = []

In [ ]:
for i in texts:
    if '돌발' in i:
        장소.append(i.split(',')[0]+i.split(',')[1])
        방향.append(i.split(',')[2])
        유형.append('사고')
    else:
        장소.append(i.split(',')[0])
        방향.append(i.split(',')[1])
        유형.append(i.split(',')[-1])
    

In [ ]:
display(len(장소))
display(len(유형))

In [ ]:
장소

In [ ]:
obj = ['<대전통영간 고속도로> 대전방향','<대전통영간 고속도로> 통영방향','<대전통영간 고속도로> 비룡분기점~산내분기점','<대전당진간 고속도로> 대전방향']

- 위의 데이터는 제거한다 또한, 트위터의 text 내용이 실시간으로 바뀌기 때문에 제거할 대상의 데이터는 코드를 확인하여 삭제

In [ ]:
filtered_장소 = []
num = 0
num_list =[]
print('-'*30)
for i in 장소:
    
    text = i
    print('{}번째 for문 시작'.format(num))
    print('텍스트는 :',text)
    
    if i in obj:
        print('해당 텍스트는 제외')
        num_list.append(num)
        num += 1
        print('-'*30)
        continue

    print('while문 시작')
    print()
    ## 텍스트의 도로명만 남기기위해 while 반복문 사용
    while True:
        a = text.find(' ')
        
        if a == 0:
            break
        elif a == -1:
            break
        ## 고속도로의 경우 텍스트의 앞쪽에 위치하여 슬라이싱을 반대로
        if '고속도로' in text:
            text = text[:a]
        elif text == '4.대덕대로 ':
            break
    
        else:
            text = text[a:]
        print('첫번째 while :',text)
    
    # 텍스트의 .을 없애기 위해 사용
    while True:
        b = text.find('.')
        if b == 0:
            break
        elif b == -1:
            break
        text = text[b+1:]
        print('두번째 while :',text)
    
    # > , < , → 을 없애기 위해 반복문 사용
    while True:
        c = text.find('>')
        if c == 0:
            break
        elif c == -1:
            break
        if '고속도로' in text:
            text = text[:c]
        else:
            text = text[c+1:]
        print('세번째 while :',text)
        
    while True:
        d = text.find('<')
        if d == -1:
            break
        if '고속도로' in text:
            text = text[d+1:]
        else:
            pass
        print('네번째 while :',text)    
    
    
    while True:
        e = text.find('→')
        if e == 0:
            break
        elif e == -1:
            break
        text = text[e+1:] 
        print('마지막 while :',text)
    
    ## 공백을 없앰
    text = text.lstrip()
    text = text.rstrip()
    
    print('최종 text :',text)
    print('-'*30)
    filtered_장소.append(text)
    num += 1
print('제외한 텍스트는 총 : {}개입니다. index 값은 다음과 같습니다.'.format(len(num_list)))
for t in num_list:
    print(t)

In [ ]:
filtered_장소

In [ ]:
len(filtered_장소)

- 언급된 text의 count 값을 구하기 위해 from collections import Counter 사용

In [ ]:
filtered_장소

In [ ]:
count = Counter(filtered_장소)

In [ ]:
count

In [ ]:
count_list = count.most_common(20)
count_list

- 언급이 가장 많이 된 20가지를 추출

In [ ]:
word = []
freq = []

In [ ]:
for i,j in count_list : # i,j 함으로써 언급단어,언급수를 가져옴
    word.append(i)
    freq.append(j)

In [ ]:
df = pd.DataFrame({'언급단어' : word , '언급수' : freq})

In [ ]:
df

In [ ]:
from wordcloud import WordCloud

In [ ]:
#wc = WordCloud(width=400, height=400, background_color='white')
# 영어 일때

wc = WordCloud(font_path='C:\Windows\Fonts\malgun.ttf', width=400, height=400, background_color='white')
# 한글 일때

In [ ]:
frequencies = {word: freq for word, freq in zip(word,freq)}
# dictionery 형태로 변경

In [ ]:
wc.fit_words(frequencies)

In [ ]:
wc.to_image()

- 대전지방경찰청 교통정보센터에 언급이 가장 많이 된 순서대로 워드클라우드

In [ ]:
wc.to_file('대전지방경찰청_교통정보센터_.png')